In [1]:
import pandas as pd
from docx import Document
import re
import pandas as pd
import torch
from transformers import BigBirdTokenizer,BigBirdModel,BigBirdForSequenceClassification
import PyPDF2
from torch import Tensor
import numpy as np
from collections import Counter

In [2]:
Data_csv = pd.read_csv("/home/ubuntu/working_directory/Bert_experimentation/new_dataframe_50000_100000.csv")
Data_csv.drop('Unnamed: 0', axis=1, inplace=True)
Data_csv

,FileName,text,Level
0,15031-4983-FullBook.docx,"Learner Choice, Learning Voice\n\n\nLearner Vo...",Level 1
1,15031-4984-FullBook.docx,\n\nExistentialism: A Philosophical Inquiry\n\...,Level 1
2,15031-4985-FullBook.docx,"""The editors of this volume are the top practi...",Level 1
3,15031-4986-FullBook.docx,Black Power Music!\n\nBlack Power Music!: Prot...,Level 1
4,15031-4994-FullBook.docx,"""A masterful and highly accessible overview of...",Level 1
...,...,...,...
690,15032-5743-FullBook.docx,Student-Centred Approaches to Russian Language...,Level 3
691,15032-5749-FullBook.docx,Ancient and Early Medieval Kingdoms of the Pam...,Level 3
692,15032-5789-FullBook.docx,The Acquisition of English Grammar and Phonolo...,Level 3
693,15032-5829-FullBook.docx,"\n‘New concepts, new words for them, new actio...",Level 3


In [6]:
sample_df = Data_csv.groupby("Level").apply(lambda x: x.sample(1))

In [7]:
sample_df

,,FileName,text,Level
Level,,,,
Level 1,104,15031-5376-FullBook.docx,Transforming Indigenous Higher Education\n\n\n...,Level 1
Level 2,622,15032-5536-FullBook.docx,"Divine Envy, Jealousy, and Vengefulness in Anc...",Level 2
Level 3,674,15031-5256-FullBook.docx,"Ancient Chinese Academy, Confucianism, and Soc...",Level 3


In [8]:
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base', 
                                                         num_labels=3)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassifica

In [9]:
def split_tokens_into_smaller_chunks(input_id: Tensor,att_mask: Tensor, chunk_size: int, stride: int, minimal_chunk_length: int):
    input_id_chunks = [input_id[i : i + chunk_size] for i in range(0, len(input_id), stride)]
    mask_chunks = [att_mask[i : i + chunk_size] for i in range(0, len(att_mask), stride)]
    if len(input_id_chunks) > 1:
        # ignore chunks with less than minimal_length number of tokens
        input_id_chunks = [x for x in input_id_chunks if len(x) >= minimal_chunk_length]
        mask_chunks = [x for x in mask_chunks if len(x) >= minimal_chunk_length]
    return input_id_chunks, mask_chunks


def add_special_tokens_at_beginning_and_end(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    """
    Adds special CLS token (token id = 101) at the beginning.
    Adds SEP token (token id = 102) at the end of each chunk.
    Adds corresponding attention masks equal to 1 (attention mask is boolean).
    """
    for i in range(len(input_id_chunks)):
        # adding CLS (token id 101) and SEP (token id 102) tokens
        input_id_chunks[i] = torch.cat([Tensor([101]), input_id_chunks[i], Tensor([102])])
        # adding attention masks  corresponding to special tokens
        mask_chunks[i] = torch.cat([Tensor([1]), mask_chunks[i], Tensor([1])])

def add_padding_tokens(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    """Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens."""
    for i in range(len(input_id_chunks)):
        # get required padding length
        pad_len = 512 - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([input_id_chunks[i], Tensor([0] * pad_len)])
            mask_chunks[i] = torch.cat([mask_chunks[i], Tensor([0] * pad_len)])
            
def stack_tokens_from_all_chunks(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> tuple[Tensor, Tensor]:
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)
    return input_ids.long(), attention_mask.int()

In [22]:
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn.functional as F
new_chunk_df = pd.DataFrame()

total_chunks = []  
total_att_mask = [] 
total_fnames = []
total_labels = []
total_chunk_counts = []
label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
chunks_count = 1
chunk_size = 510 
stride = 256
minimal_chunk_length = 256
for idx in range(sample_df.shape[0]):
    x = sample_df["text"].iloc[idx]
    sentences = re.split(r'(?<=[.!?])\s+', x)
    long_string = ' '.join(sentences)
    label = label2id[sample_df.iloc[idx]["Level"]]
    # one_hot_tensor = F.one_hot(torch.tensor(label), num_classes=len(label2id)).float()
    # print(one_hot_tensor)
    # label_vec = pd.get_dummies([label], columns=label2id.values()).values.squeeze()
    # label_tensor = torch.tensor(label_vec, dtype=torch.float32)
    tokens = tokenizer(long_string, add_special_tokens=True, truncation=False, return_tensors="pt")
    input_id_chunks, mask_chunks = split_tokens_into_smaller_chunks(tokens["input_ids"][0],tokens["attention_mask"][0], chunk_size, stride, minimal_chunk_length)
    add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks)
    add_padding_tokens(input_id_chunks, mask_chunks)
    label_vec = [label] * len(input_id_chunks)
    # print(len(mask_chunks))
    # print(len(input_id_chunks))
    # print(label_tensor)
    total_chunks.extend((input_id_chunks))
    total_att_mask.extend((mask_chunks))
    total_labels.extend((label_vec))

    # total_labels.
    # input_ids, attention_mask = stack_tokens_from_all_chunks(input_id_chunks, mask_chunks)
    
    

In [23]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels_vec):
        self.input_ids = [chunk.long() for chunk in input_ids]
        self.attention_mask = attention_mask
        self.labels_vec = labels_vec
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        print(self.labels_vec[index])
        return {
            'input_ids': self.input_ids[index].squeeze(),
            'attention_mask': self.attention_mask[index].squeeze(),
            'label': self.labels_vec[index]
        }

In [24]:
dataset =  CustomDataset(total_chunks,total_att_mask,total_labels)

In [25]:
from torch.utils.data import random_split
train_ratio = 0.8  # 80% for training
test_ratio = 0.2  # 20% for testing

num_samples = len(dataset)
train_size = int(train_ratio * num_samples)
test_size = num_samples - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [26]:
len(train_dataset)

1203

In [27]:
train_dataset[0]

0


{'input_ids': tensor([  101,   112, 18092,   391, 30674,  7287,   357,  2697, 45314,  1835,
         20411, 14750,   458,  3054, 26131,  2224,   536,  2047,   838,  1419,
           419,   382,  7768,  9386,   427, 28126,   363, 12852,   387,  1212,
          7734,  5713,   391,  3808,  4889,   391,  3458,   430, 26533,   762,
           458,  3957,   512,  3942,  2224,   536,  1640,  2422,   127, 28749,
           112,  2422,   127, 22512, 12100,   112,  1685,   127, 28377,  2224,
           536,  1640,  2965,   127,  2083,  3204,   464, 12215,   112,  7896,
           127, 48785,   112,  1685,   838,  1182,  4474,   458, 23445,   109,
          4811,   100, 22106, 24134,   113, 18244,  3808,  2551,   388,  4606,
           112,  9957, 12892,   569,   688,  3588,  1518,   933,   363, 10140,
           365, 44756,  8233,   523,   363,   321,   100, 49888,   100,  1530,
           456,  9444,   517, 24713,   644,  2393, 21508,   456,   382, 11620,
           737,   387,  2376, 47204,  1

In [ ]:
# train_data = pd.read_parquet('/home/ubuntu/working_directory/Bert_experimentation/80_train.parquet')
# val_df = pd.read_parquet('/home/ubuntu/working_directory/Bert_experimentation/80_val.parquet')
class CustomDataset(Dataset):
    def __init__(self, data, label2id, tokenizer):
        self.data = data.sample(4000)
        self.label2id = label2id
        self.tokenizer= tokenizer
        self.labels = self.data["labels"].apply(lambda x:self.label2id[x]).to_list()
        self.one_hot_labels = pd.get_dummies(self.labels)
        self.one_hot_labels = torch.tensor(np.array(self.one_hot_labels), dtype=torch.float)
        print("Data:",self.data.labels.value_counts())
        print("counter:",Counter(self.labels))
        # print("@@@@",len(self.data))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text = self.data.iloc[index]['texts']

        # print("text::",index,text)
        # print('\n')
        label = self.one_hot_labels[index]
        # print("original length:", len(text.split(" ")), len(text))
        encoded_text = self.tokenizer.encode_plus(
            str(text),
            max_length=512,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {'input_ids': encoded_text['input_ids'].squeeze(),
                 'attention_mask': encoded_text['attention_mask'].squeeze(),
                 'label':label}   
                 
                 
label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
old_train_dataset = CustomDataset(train_data,label2id,tokenizer)
old_test_dataset = CustomDataset(val_df,label2id,tokenizer)


In [ ]:
len(old_train_dataset)
print(old_test_dataset[0])

In [28]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=2,
                                           pin_memory=True,
                                           shuffle=False, 
                                           )
test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=3,
                                           pin_memory=True,
                                           )

In [17]:
model

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [16]:
for i in train_loader:
    print(i)

tensor([1., 0., 0.])
tensor([1., 0., 0.])
{'input_ids': tensor([[  101,    65,   321,  ...,  3477,  2544,   102],
        [  101, 33316,  3884,  ...,  1898, 15867,   102]]), 'attention_mask': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'label': tensor([[1., 0., 0.],
        [1., 0., 0.]])}
tensor([1., 0., 0.])
tensor([1., 0., 0.])
{'input_ids': tensor([[  101,  3771,   391,  ...,   864,  1057,   102],
        [  101,   113,   116,  ..., 27989,   388,   102]]), 'attention_mask': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'label': tensor([[1., 0., 0.],
        [1., 0., 0.]])}
tensor([1., 0., 0.])
tensor([1., 0., 0.])
{'input_ids': tensor([[  101,   864, 34828,  ...,  5481,   385,   102],
        [  101,  3906,   363,  ...,   385,   880,   102]]), 'attention_mask': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'label': tensor([[1., 0., 0.],
        [1., 0., 0.]])}
tensor([1., 0

KeyboardInterrupt: 

In [29]:

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

EPOCHS = 5
LEARNING_RATE = 0.0000025 ######
BATCH_SIZE = 2
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.04) #####
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=0.04)

scheduler = get_linear_schedule_with_warmup(optimizer, 
             num_warmup_steps=50, ########
            num_training_steps=len(train_loader)*EPOCHS )   

In [30]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")


In [31]:
from torch.nn.utils import clip_grad_norm_
# from tqdm import tqdm
from tqdm.notebook import tqdm

import numpy as np
import math

train_loss_per_epoch = []
val_loss_per_epoch = []

model = model.to(device)
for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_loader,desc='Training')):
        # tqdm_desc = f'Training ({step_num+1}/{train_loader_length})'
        # print("batch>>>",batch_data)
        input_ids, att_mask, labels = batch_data["input_ids"].to(device),batch_data["attention_mask"].to(device),batch_data["label"].to(device)
        input_ids = input_ids.to(torch.long).to(device)
        att_mask = att_mask.to(torch.long).to(device)
        labels = labels.to(torch.long).to(device)
        print(input_ids.shape,labels.shape)
        # input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)
        # print("logits***:", output["logits"])
        
        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))    


    # '''
    # Validation
    # '''
    # model.eval()
    # valid_loss = 0
    # valid_pred = []
    # with torch.no_grad():
    #     for step_num_e, batch_data in enumerate(tqdm(test_loader,desc='Validation')):
            
    #         input_ids, att_mask, labels = batch_data["input_ids"].to(device),batch_data["attention_mask"].to(device),batch_data["label"].to(device)
    #         # input_ids, att_mask, labels = [data.to(device) for data in batch_data]
    #         output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)
    #         print("logits***:", output["logits"])

    #         loss = output.loss
    #         valid_loss += loss.item()

    #         valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
        
    # val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    # valid_pred = np.concatenate(valid_pred)

    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(val_df) / BATCH_SIZE), train_loss / (step_num + 1)))
    # print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(val_df) / BATCH_SIZE), valid_loss / (step_num_e + 1)))          


Epoch:  1


Training:   0%|          | 0/602 [00:00<?, ?it/s]

0
2
torch.Size([2, 512]) torch.Size([2])
2
1
torch.Size([2, 512]) torch.Size([2])
0
1
torch.Size([2, 512]) torch.Size([2])
2
0
torch.Size([2, 512]) torch.Size([2])
0
2
torch.Size([2, 512]) torch.Size([2])
0
1
torch.Size([2, 512]) torch.Size([2])
2
0
torch.Size([2, 512]) torch.Size([2])
2
2
torch.Size([2, 512]) torch.Size([2])
2
1
torch.Size([2, 512]) torch.Size([2])
0
1
torch.Size([2, 512]) torch.Size([2])
0
1
torch.Size([2, 512]) torch.Size([2])
2
0
torch.Size([2, 512]) torch.Size([2])
0
0
torch.Size([2, 512]) torch.Size([2])
1
0
torch.Size([2, 512]) torch.Size([2])
0
2
torch.Size([2, 512]) torch.Size([2])
0
1
torch.Size([2, 512]) torch.Size([2])
2
0
torch.Size([2, 512]) torch.Size([2])
0
2
torch.Size([2, 512]) torch.Size([2])
0
2
torch.Size([2, 512]) torch.Size([2])
1
0
torch.Size([2, 512]) torch.Size([2])
0
0
torch.Size([2, 512]) torch.Size([2])
1
2
torch.Size([2, 512]) torch.Size([2])
1
0
torch.Size([2, 512]) torch.Size([2])
0
1
torch.Size([2, 512]) torch.Size([2])
0
2
torch.Size([

KeyboardInterrupt: 

: 

In [ ]:
import matplotlib.pyplot as plt

# Rest of your code...

epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(8, 8))
plt.plot(epochs, train_loss_per_epoch, label='training loss')
plt.plot(epochs, val_loss_per_epoch, label='validation loss')
plt.title("Training and Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Save the plot as a PNG file
plt.savefig('./loss_plotBig_bird_new_data.png')


In [ ]:
valid_true = [batch["label"].detach().cpu().numpy() for batch in test_loader]
valid_true = np.concatenate(valid_true)
valid_true = np.argmax(valid_true, axis=1)

In [ ]:
print(classification_report(valid_pred, valid_true, target_names= ["level 1","level 2","level 3"]))